## Placeholder for Table of Contents Bug on AWS

## Placeholder for Table of Contents Bug on AWS

## Placeholder for Table of Contents Bug on AWS

## Placeholder for Table of Contents Bug on AWS

# Imports

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
import ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.metrics import f1_score
from imblearn.under_sampling import RandomUnderSampler
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
from pprint import pprint

import gc
import optuna
from sklearn.feature_selection import RFE
from math import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.activations import tanh
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation,Dense,Dropout, Conv2D, Concatenate, LeakyReLU, MaxPooling2D, Reshape, Input, LSTM

/tmp/ipykernel_16629/3029808958.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-12 23:21:16.133488: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 23:21:16.172532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AV

In [2]:
df = pd.read_csv('/home/ec2-user/Capstone/pair_features1_300_15.csv')
df.head()

,Date,Ticker_P1,Close_P1,Ticker_P2,Close_P2,High_P1,High_P2,Low_P1,Low_P2,Volume_P1,...,abs_spread_normed_max,abs_spread_normed_90th,abs_spread_normed_75th,abs_spread_normed_median,abs_spread_normed_l7_avg,abs_spread_normed_l14_avg,cos_sim,corr_coef,pnls,num_entries
0,2015-01-02,SLB,66.153,GRMN,39.250,86.200,53.140,84.620,52.320,5500800.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-05,SLB,64.361,GRMN,38.167,84.400,52.410,82.590,51.080,10039400.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-06,SLB,63.103,GRMN,37.705,83.360,51.250,80.660,50.080,9758400.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-07,SLB,63.095,GRMN,38.406,82.660,51.980,80.530,51.340,7951600.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-08,SLB,63.859,GRMN,38.406,83.290,51.870,81.940,50.960,7911600.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
sector_df = pd.read_csv('/home/ec2-user/Capstone/1999-12-01-2023-12-31.csv')

In [4]:
print(df.shape)
sector_df = sector_df[['Date','Ticker','Open','Close','Adj Close']]
sector_df.columns = ['Date','Ticker_P1','Open_P1','Close_P1','Adj Close_P1']
df = pd.merge(df,sector_df,\
                     on=['Date','Ticker_P1'],how='left',suffixes=[None,'_y'])
print(df.shape)
sector_df.columns = ['Date','Ticker_P2','Open_P2','Close_P2','Adj Close_P2']
df = pd.merge(df,sector_df,\
                     on=['Date','Ticker_P2'],how='left',suffixes=[None,'_y'])
df.shape

(1257000, 27)
(1257000, 30)


(1257000, 33)

In [5]:
df['pnls'] = df['pnls']*100.0

In [6]:
total_days = df.Date.nunique()
total_days

1257

In [7]:
print(df.shape)
df.isna().sum()

(1257000, 33)


Date                              0
Ticker_P1                         0
Close_P1                          0
Ticker_P2                         0
Close_P2                          0
High_P1                           0
High_P2                           0
Low_P1                            0
Low_P2                            0
Volume_P1                         0
Volume_P2                         0
abs_spread                        0
abs_spread_mean              299000
abs_spread_std               299000
abs_spread_mean_l28           27000
abs_spread_std_l28            27000
spread_normed                299000
abs_spread_normed_max        598000
abs_spread_normed_90th       598000
abs_spread_normed_75th       598000
abs_spread_normed_median     598000
abs_spread_normed_l7_avg     305000
abs_spread_normed_l14_avg    312000
cos_sim                      299000
corr_coef                    299000
pnls                         315000
num_entries                  315000
Open_P1                     

In [8]:
df.Date.nunique()

1257

In [9]:
df.abs_spread_normed_max.dropna().describe()

count   659000.000
mean         3.219
std          0.743
min          1.096
25%          2.718
50%          3.138
75%          3.578
max          7.090
Name: abs_spread_normed_max, dtype: float64

In [10]:
tmp_df = df[(df.Ticker_P1 == 'TDY') & (df.Ticker_P2 == 'PEP')]
tmp_df.head()

,Date,Ticker_P1,Close_P1,Ticker_P2,Close_P2,High_P1,High_P2,Low_P1,Low_P2,Volume_P1,...,cos_sim,corr_coef,pnls,num_entries,Open_P1,Close_P1_y,Adj Close_P1,Open_P2,Close_P2_y,Adj Close_P2


In [11]:
tmp_df.abs_spread_normed_max

Series([], Name: abs_spread_normed_max, dtype: float64)

In [12]:
df = df.sort_values('Date')

In [13]:
print(df.Date.min(),df.Date.max())

2015-01-02 2019-12-30


# Bad Data

In [14]:
print(df.shape)
df = df[df['Volume_P1']>0]
print(df.shape)
df = df[df['Volume_P2']>0]
print(df.shape)


(1257000, 33)
(1257000, 33)
(1255183, 33)


In [15]:
df = df[df['abs_spread']>=1]
print(df.shape)

(1232286, 33)


In [16]:
df.tail(2)

,Date,Ticker_P1,Close_P1,Ticker_P2,Close_P2,High_P1,High_P2,Low_P1,Low_P2,Volume_P1,...,cos_sim,corr_coef,pnls,num_entries,Open_P1,Close_P1_y,Adj Close_P1,Open_P2,Close_P2_y,Adj Close_P2
843446,2019-12-30,GRMN,88.556,ENPH,25.370,98.630,26.850,97.630,25.230,429500.000,...,0.909,0.625,NaN,NaN,98.470,97.950,88.556,26.710,25.370,25.370
1256999,2019-12-30,TYL,298.360,NWS,13.808,300.900,14.500,294.780,14.410,300500.000,...,0.993,0.633,NaN,NaN,300.160,298.360,298.360,14.420,14.460,13.808


# Label

In [17]:
# df['abs_spread_mean'] = df['abs_spread_mean_l20']

for i in range(17,20):
    df['FUTURE_abs_spread_'+str(i)] = df.groupby(['Ticker_P1','Ticker_P2'])['abs_spread'].shift(-1*i)

df['FUTURE_abs_spread_avg'] = df[[c for c in df.columns if 'FUTURE_abs_spread_' in c]].mean(axis=1)

df['PCT_CHANGE'] = (df['FUTURE_abs_spread_avg'] - df['abs_spread'])*100.0/df['abs_spread']
# df['direction'] = np.where(df['PCT_CHANGE'].abs()<3,0,np.where(df['PCT_CHANGE']>0,1,-1))
# df['direction'] = np.where((df['FUTURE_abs_spread_avg'] - (df['abs_spread_mean']+1.5*df['abs_spread_std']))>0,1,0)
df['direction'] = np.where((df['FUTURE_abs_spread_avg'] - (df['abs_spread_mean']+0.1*df['abs_spread_std']))<0,1,0)

df['direction'].value_counts()
# df['direction'] = df['PCT_CHANGE']

direction
0    867089
1    365197
Name: count, dtype: int64

# Feature Engineering

In [18]:
all_pairs = df[['Ticker_P1','Ticker_P2']].drop_duplicates()
all_pairs

,Ticker_P1,Ticker_P2
0,SLB,GRMN
913839,GRMN,PODD
87990,SLB,EXR
915096,GRMN,WBA
916353,GRMN,CCI
...,...,...
789400,GRMN,GIS
926415,GRMN,ADM
903789,GRMN,LUV
897506,GRMN,L


In [19]:
for c in ['Open','Low','High']:
    for p in ['1','2']:
        df[c+'_P'+p] = np.where(df['Adj Close_P'+p]!=df['Close_P'+p+'_y'],\
                                df['Adj Close_P'+p]*df[c+'_P'+p]/df['Close_P'+p+'_y'],\
                                df[c+'_P'+p])

In [21]:
for i in range(301,df.Date.nunique()-120):
    for ticker1, ticker2 in all_pairs.values:
        data1 = df[(df.Ticker_P1 == ticker1)][['Date','Close_P1','Open_P1','Low_P1','High_P1']].drop_duplicates()
        data1['SMA100'] = data1['Close_P1'].rolling(7).mean()
        data1 = data1.iloc[i:i+300]
        data1.columns = ['Date','Close','Open','Low','High','SMA100']
        data1

        data2 = df[(df.Ticker_P2 == ticker2)][['Date','Close_P2','Open_P2','Low_P2','High_P2']].drop_duplicates()
        data2['SMA100'] = data2['Close_P2'].rolling(7).mean()
        data2 = data2.iloc[i:i+300]
        data2.columns = ['Date','Close','Open','Low','High','SMA100']
        data2

        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(go.Candlestick(x=data1['Date'],
                                             open=data1['Open'],
                                             high=data1['High'],
                                             low=data1['Low'],
                                             close=data1['Close'],name='t1',opacity=0.7), secondary_y=False)

        fig.add_trace(go.Candlestick(x=data2['Date'],
                                             open=data2['Open'],
                                             high=data2['High'],
                                             low=data2['Low'],
                                             close=data2['Close'],\
                                             increasing_line_color= 'purple', decreasing_line_color= 'cyan',\
                                                name='t2',opacity=0.7), secondary_y=True)
        fig.add_trace(go.Scatter(x=data1['Date'],
                         y=data1["SMA100"],
                         name="SMA1"
                        ), secondary_y=False)
        fig.add_trace(go.Scatter(x=data2['Date'],
                         y=data2["SMA100"],
                         name="SMA2"
                        ), secondary_y=True)
        fig.update_layout(xaxis_rangeslider_visible=False)
        print(ticker1+'_'+ticker2+'_'+data1.Date.iloc[-1])
        f_name = ticker1+'_'+ticker2+'_'+data1.Date.iloc[-1]
        fig.write_image("/home/ec2-user/Capstone/Image/"+f_name+".png") 
#         fig.show()
#         break
#     break

SLB_GRMN_2017-05-22
GRMN_PODD_2017-05-22
SLB_EXR_2017-05-22
GRMN_WBA_2017-05-22
GRMN_CCI_2017-05-22
GRMN_WFC_2017-05-22
GRMN_CHTR_2017-05-22
SLB_MCHP_2017-05-22
GRMN_NDSN_2017-05-22
GRMN_HOLX_2017-05-22
GRMN_ZBH_2017-05-22
GRMN_ADP_2017-05-22
SLB_RTX_2017-05-22
GRMN_PPG_2017-05-22
GRMN_DTE_2017-05-22
GRMN_CRM_2017-05-22
SLB_VMC_2017-05-22
GRMN_POOL_2017-05-22
GRMN_RHI_2017-05-22
GRMN_EA_2017-05-22
SLB_SHW_2017-05-22
GRMN_GNRC_2017-05-22
GRMN_ELV_2017-05-22
GRMN_URI_2017-05-22
GRMN_HBAN_2017-05-22
SLB_CAH_2017-05-22
GRMN_FITB_2017-05-22
GRMN_XOM_2017-05-22
GRMN_JBHT_2017-05-22
GRMN_NCLH_2017-05-22
SLB_CPT_2017-05-22
GRMN_EL_2017-05-22
GRMN_UNP_2017-05-22
SLB_EQT_2017-05-22
GRMN_DXCM_2017-05-22
GRMN_ALLE_2017-05-22
GRMN_GE_2017-05-22
GRMN_PNW_2017-05-22
SLB_CHD_2017-05-22
GRMN_STT_2017-05-22
GRMN_IVZ_2017-05-22
GRMN_DUK_2017-05-22
GRMN_SBAC_2017-05-22
SLB_ADBE_2017-05-22
GRMN_XEL_2017-05-22
GRMN_CNP_2017-05-22
GRMN_PRU_2017-05-22
SLB_CB_2017-05-22
GRMN_CFG_2017-05-22
GRMN_HRL_2017-05-22


GRMN_ROST_2017-05-22
GRMN_EXC_2017-05-22
GRMN_LKQ_2017-05-22
SLB_LRCX_2017-05-22
GRMN_PH_2017-05-22
GRMN_EXPE_2017-05-22
GRMN_LHX_2017-05-22
SLB_HST_2017-05-22
GRMN_GLW_2017-05-22
SLB_MRO_2017-05-22
GRMN_VTR_2017-05-22
GRMN_MSFT_2017-05-22
GRMN_ARE_2017-05-22
GRMN_PXD_2017-05-22
SLB_CVX_2017-05-22
GRMN_GWW_2017-05-22
GRMN_ALB_2017-05-22
GRMN_KR_2017-05-22
GRMN_IFF_2017-05-22
SLB_DE_2017-05-22
GRMN_IDXX_2017-05-22
GRMN_WTW_2017-05-22
GRMN_PARA_2017-05-22
GRMN_PNR_2017-05-22
SLB_HIG_2017-05-22
GRMN_IQV_2017-05-22
GRMN_VTRS_2017-05-22
GRMN_D_2017-05-22
SLB_DHI_2017-05-22
GRMN_INTC_2017-05-22
GRMN_CMCSA_2017-05-22
GRMN_HUBB_2017-05-22
SLB_C_2017-05-22
GRMN_RL_2017-05-22
GRMN_TGT_2017-05-22
GRMN_MPWR_2017-05-22
GRMN_JKHY_2017-05-22
GRMN_HAL_2017-05-22
GRMN_WYNN_2017-05-22
GRMN_KIM_2017-05-22
SLB_ICE_2017-05-22
GRMN_PHM_2017-05-22
GRMN_GILD_2017-05-22
GRMN_SRE_2017-05-22
GRMN_ZBRA_2017-05-22
SLB_SJM_2017-05-22
GRMN_CF_2017-05-22
GRMN_COST_2017-05-22
GRMN_AMP_2017-05-22
GRMN_QCOM_2017-05-22
S

SLB_VZ_2017-05-22
GRMN_T_2017-05-22
GRMN_ZTS_2017-05-22
GRMN_REGN_2017-05-22
GRMN_PG_2017-05-22
SLB_PWR_2017-05-22
GRMN_REG_2017-05-22
GRMN_SO_2017-05-22
GRMN_KEYS_2017-05-22
SLB_NEM_2017-05-22
GRMN_CAG_2017-05-22
SLB_PEP_2017-05-22
GRMN_ALGN_2017-05-22
SLB_PAYX_2017-05-22
SLB_NI_2017-05-22
SLB_LLY_2017-05-22
SLB_BEN_2017-05-22
SLB_SWKS_2017-05-22
SLB_WAT_2017-05-22
SLB_MTB_2017-05-22
SLB_CTLT_2017-05-22
SLB_PKG_2017-05-22
SLB_TAP_2017-05-22
SLB_MAS_2017-05-22
SLB_CNC_2017-05-22
SLB_UNH_2017-05-22
SLB_BBWI_2017-05-22
SLB_EXPD_2017-05-22
SLB_MCO_2017-05-22
SLB_APA_2017-05-22
SLB_ISRG_2017-05-22
SLB_TER_2017-05-22
SLB_NTAP_2017-05-22
SLB_LYV_2017-05-22
SLB_MCK_2017-05-22
SLB_NVDA_2017-05-22
SLB_AON_2017-05-22
SLB_CBOE_2017-05-22
SLB_TJX_2017-05-22
SLB_ROP_2017-05-22
SLB_WY_2017-05-22
SLB_PGR_2017-05-22
SLB_EMN_2017-05-22
SLB_CDNS_2017-05-22
SLB_ARE_2017-05-22
SLB_VRSN_2017-05-22
SLB_PXD_2017-05-22
SLB_GWW_2017-05-22
SLB_ALB_2017-05-22
SLB_KO_2017-05-22
SLB_KR_2017-05-22
SLB_IFF_2017-05-2

GRMN_LYB_2017-05-23
GRMN_LVS_2017-05-23
GRMN_AMGN_2017-05-23
GRMN_EG_2017-05-23
SLB_BA_2017-05-23
GRMN_TSLA_2017-05-23
GRMN_PFG_2017-05-23
GRMN_ITW_2017-05-23
GRMN_MAR_2017-05-23
SLB_WDC_2017-05-23
GRMN_HES_2017-05-23
GRMN_ILMN_2017-05-23
GRMN_TDG_2017-05-23
GRMN_PPL_2017-05-23
SLB_DLTR_2017-05-23
GRMN_PGR_2017-05-23
GRMN_CL_2017-05-23
GRMN_ROP_2017-05-23
GRMN_TJX_2017-05-23
GRMN_MKTX_2017-05-23
GRMN_EMN_2017-05-23
GRMN_PAYX_2017-05-23
SLB_GD_2017-05-23
GRMN_NI_2017-05-23
GRMN_LLY_2017-05-23
GRMN_BEN_2017-05-23
GRMN_WAT_2017-05-23
SLB_FFIV_2017-05-23
GRMN_MTB_2017-05-23
GRMN_CTLT_2017-05-23
GRMN_PKG_2017-05-23
GRMN_MAS_2017-05-23
SLB_FDS_2017-05-23
GRMN_CNC_2017-05-23
GRMN_UNH_2017-05-23
GRMN_BBWI_2017-05-23
GRMN_MCO_2017-05-23
SLB_EOG_2017-05-23
GRMN_APA_2017-05-23
GRMN_ISRG_2017-05-23
GRMN_TER_2017-05-23
GRMN_LYV_2017-05-23
SLB_META_2017-05-23
GRMN_MCK_2017-05-23
GRMN_NVDA_2017-05-23
GRMN_CBOE_2017-05-23
SLB_CHRW_2017-05-23
GRMN_STLD_2017-05-23
GRMN_MMM_2017-05-23
TYL_DFS_2017-05-23


SLB_HPQ_2017-05-23
SLB_FE_2017-05-23
SLB_FI_2017-05-23
SLB_FLT_2017-05-23
SLB_GOOGL_2017-05-23
SLB_CSCO_2017-05-23
SLB_AWK_2017-05-23
SLB_BSX_2017-05-23
SLB_BLDR_2017-05-23
SLB_PNC_2017-05-23
SLB_LULU_2017-05-23
SLB_DG_2017-05-23
SLB_SYF_2017-05-23
SLB_EIX_2017-05-23
SLB_DIS_2017-05-23
SLB_BKR_2017-05-23
SLB_AIZ_2017-05-23
SLB_EQIX_2017-05-23
SLB_VRTX_2017-05-23
SLB_BIIB_2017-05-23
SLB_SYK_2017-05-23
SLB_KMI_2017-05-23
SLB_CRL_2017-05-23
SLB_V_2017-05-23
SLB_CE_2017-05-23
SLB_CCI_2017-05-23
SLB_WFC_2017-05-23
SLB_MAA_2017-05-23
SLB_CHTR_2017-05-23
SLB_NDSN_2017-05-23
SLB_HOLX_2017-05-23
SLB_HD_2017-05-23
SLB_ZBH_2017-05-23
SLB_ADP_2017-05-23
SLB_PPG_2017-05-23
SLB_ADM_2017-05-23
SLB_NXPI_2017-05-23
SLB_DTE_2017-05-23
SLB_GIS_2017-05-23
SLB_CRM_2017-05-23
SLB_RHI_2017-05-23
SLB_HII_2017-05-23
SLB_EA_2017-05-23
SLB_GNRC_2017-05-23
SLB_ELV_2017-05-23
SLB_URI_2017-05-23
SLB_IT_2017-05-23
SLB_HBAN_2017-05-23
SLB_FITB_2017-05-23
SLB_JBHT_2017-05-23
SLB_FAST_2017-05-23
SLB_AMZN_2017-05-23
SLB

...

SLB_LKQ_2017-05-24
SLB_PH_2017-05-24
SLB_CSX_2017-05-24
SLB_EXPE_2017-05-24
SLB_DAL_2017-05-24
SLB_NVR_2017-05-24
SLB_STE_2017-05-24
SLB_CAT_2017-05-24
SLB_APD_2017-05-24
SLB_EFX_2017-05-24
SLB_SPG_2017-05-24
SLB_COO_2017-05-24
SLB_BG_2017-05-24
SLB_MGM_2017-05-24
SLB_TECH_2017-05-24
SLB_DOV_2017-05-24
SLB_FRT_2017-05-24
SLB_BAX_2017-05-24
SLB_IEX_2017-05-24
SLB_NSC_2017-05-24
SLB_ODFL_2017-05-24
SLB_DVA_2017-05-24
SLB_CTAS_2017-05-24
SLB_UHS_2017-05-24
SLB_UPS_2017-05-24
SLB_KLAC_2017-05-24
SLB_COR_2017-05-24
SLB_IBM_2017-05-24
SLB_AMAT_2017-05-24
SLB_SCHW_2017-05-24
SLB_ALL_2017-05-24
SLB_BR_2017-05-24
SLB_MHK_2017-05-24
SLB_NOC_2017-05-24
SLB_NTRS_2017-05-24
SLB_PLD_2017-05-24
SLB_INTU_2017-05-24
SLB_AJG_2017-05-24
SLB_MSI_2017-05-24
SLB_HON_2017-05-24
SLB_VFC_2017-05-24
SLB_GILD_2017-05-24
SLB_SRE_2017-05-24
SLB_F_2017-05-24
SLB_ZBRA_2017-05-24
SLB_CF_2017-05-24
SLB_COST_2017-05-24
SLB_AMP_2017-05-24
SLB_GL_2017-05-24
SLB_QCOM_2017-05-24
SLB_GEN_2017-05-24
SLB_TFC_2017-05-24
SLB_GP

SLB_ADI_2017-05-24
GRMN_CHRW_2017-05-24
GRMN_META_2017-05-24
GRMN_EOG_2017-05-24
GRMN_FDS_2017-05-24
SLB_FTNT_2017-05-24
GRMN_FFIV_2017-05-24
GRMN_GD_2017-05-24
GRMN_MLM_2017-05-24
GRMN_FANG_2017-05-24
GRMN_DFS_2017-05-24
GRMN_TYL_2017-05-24
SLB_AXON_2017-05-24
SLB_KEY_2017-05-24
SLB_CZR_2017-05-24
SLB_TTWO_2017-05-24
SLB_ZION_2017-05-24
SLB_BXP_2017-05-24
SLB_WRB_2017-05-24
SLB_QRVO_2017-05-24
SLB_BKNG_2017-05-24
SLB_LYB_2017-05-24
SLB_LVS_2017-05-24
SLB_AMGN_2017-05-24
SLB_EMR_2017-05-24
GRMN_C_2017-05-24
SLB_EG_2017-05-24
SLB_PFG_2017-05-24
SLB_ITW_2017-05-24
SLB_PTC_2017-05-24
SLB_MAR_2017-05-24
SLB_HES_2017-05-24
SLB_ILMN_2017-05-24
SLB_TDG_2017-05-24
SLB_MPC_2017-05-24
SLB_PPL_2017-05-24
SLB_CL_2017-05-24
SLB_STLD_2017-05-24
SLB_MMM_2017-05-24
SLB_GOOG_2017-05-24
SLB_TSLA_2017-05-24
TYL_ROK_2017-05-24
SLB_BX_2017-05-24
TYL_NWS_2017-05-24
TYL_CDW_2017-05-24
GRMN_DRI_2017-05-24
SLB_TRGP_2017-05-24
SLB_UAL_2017-05-24
GRMN_COO_2017-05-24
SLB_AMD_2017-05-24
GRMN_CINF_2017-05-24
GRMN_A

TYL_RJF_2017-05-25
TYL_ETR_2017-05-25
GRMN_CDNS_2017-05-25
TYL_FICO_2017-05-25
SLB_NOW_2017-05-25
TYL_WDC_2017-05-25
TYL_BA_2017-05-25
TYL_RSG_2017-05-25
SLB_PAYC_2017-05-25
TYL_AEE_2017-05-25
TYL_BRO_2017-05-25
TYL_CHRW_2017-05-25
TYL_META_2017-05-25
SLB_TRV_2017-05-25
TYL_EOG_2017-05-25
TYL_FDS_2017-05-25
TYL_FFIV_2017-05-25
TYL_DLTR_2017-05-25
GRMN_PEP_2017-05-25
SLB_ALGN_2017-05-25
GRMN_APH_2017-05-25
GRMN_CAT_2017-05-25
GRMN_HON_2017-05-25
GRMN_MET_2017-05-25
SLB_PEG_2017-05-25
GRMN_CSGP_2017-05-25
GRMN_KDP_2017-05-25
GRMN_MSCI_2017-05-25
GRMN_WAB_2017-05-25
SLB_CPRT_2017-05-25
GRMN_MU_2017-05-25
GRMN_VRSK_2017-05-25
GRMN_ACGL_2017-05-25
GRMN_MTCH_2017-05-25
SLB_TFX_2017-05-25
GRMN_TSCO_2017-05-25
GRMN_BLK_2017-05-25
GRMN_ES_2017-05-25
GRMN_IP_2017-05-25
SLB_EW_2017-05-25
GRMN_TXN_2017-05-25
GRMN_ACN_2017-05-25
GRMN_TXT_2017-05-25
GRMN_MS_2017-05-25
SLB_BIO_2017-05-25
GRMN_FSLR_2017-05-25
GRMN_AAPL_2017-05-25
SLB_CBRE_2017-05-25
GRMN_CMI_2017-05-25
GRMN_NVR_2017-05-25
GRMN_CSX_201

SLB_UDR_2017-05-25
GRMN_MRO_2017-05-25
GRMN_PFE_2017-05-25
GRMN_CLX_2017-05-25
GRMN_PEAK_2017-05-25
SLB_CAG_2017-05-25
GRMN_SJM_2017-05-25
GRMN_HST_2017-05-25
SLB_MRK_2017-05-25
GRMN_BIO_2017-05-25
GRMN_EW_2017-05-25
GRMN_TFX_2017-05-25
GRMN_CPRT_2017-05-25
SLB_BALL_2017-05-25
GRMN_PEG_2017-05-25
GRMN_CBRE_2017-05-25
GRMN_LRCX_2017-05-25
SLB_ANET_2017-05-25
GRMN_HLT_2017-05-25
GRMN_HAS_2017-05-25
GRMN_BDX_2017-05-25
GRMN_ROK_2017-05-25
GRMN_CDW_2017-05-25
GRMN_AVGO_2017-05-25
GRMN_EPAM_2017-05-25
GRMN_MKC_2017-05-25
SLB_ORLY_2017-05-25
GRMN_AIG_2017-05-25
GRMN_WHR_2017-05-25
GRMN_ORLY_2017-05-25
SLB_WHR_2017-05-25
GRMN_PWR_2017-05-25
GRMN_VZ_2017-05-25
GRMN_NEM_2017-05-25
GRMN_PSA_2017-05-25
SLB_AIG_2017-05-25
GRMN_BBY_2017-05-25
GRMN_DGX_2017-05-25
GRMN_SWK_2017-05-25
GRMN_SYY_2017-05-25
SLB_MKC_2017-05-25
GRMN_AZO_2017-05-25
GRMN_LIN_2017-05-25
GRMN_DD_2017-05-25
SLB_CINF_2017-05-25
GRMN_AXP_2017-05-25
GRMN_ABT_2017-05-25
GRMN_ANET_2017-05-25
GRMN_BALL_2017-05-25
SLB_TT_2017-05-25
GR

KeyboardInterrupt: 

In [ ]:
STOP

In [ ]:
df = df[['Date', 'Ticker_P1', 'Ticker_P2', 'Close_P1', 'Volume_P1', 'Close_P2', 'Volume_P2','pnls', 'num_entries']]

In [ ]:
df = df.sort_values('Date')

In [ ]:
history_window = 120

In [ ]:
for i in tqdm(range(1,history_window+1)):
    for c in ['Close_P1','Volume_P1', 'Close_P2','Volume_P2']:
        df['PREV_'+str(i)+'_'+c] = df.groupby(['Ticker_P1','Ticker_P2'])[c].shift(i)

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
print(df.shape,df.Date.nunique())
df = df.dropna()
print(df.shape,df.Date.nunique())


In [ ]:
features = [c for c in df.columns for j in ['Close_P1','Volume_P1','Close_P2', 'Volume_P2'] if j in c]
features

# Regression

## Remove 0s?

In [ ]:
df[df.pnls==0].shape[0]*100.0/df.shape[0]

In [ ]:
df = df[df.pnls!=0]


## X,y split

In [ ]:
# df['DIFF_pnls'] = df['pnls'] - df['PREV_1_pnls']

In [ ]:
label = 'pnls'


In [ ]:
X = df[features].values
y = df[label]

In [ ]:
print(X.shape)

In [ ]:
# y = pd.Series(np.where(y.abs()<1,0,np.where(y<0,-1,1)))
y = pd.Series(np.where(y>=4,1,0))

## Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42,shuffle=False)

In [ ]:
# undersample = RandomUnderSampler(sampling_strategy='not minority')
undersample = RandomUnderSampler(sampling_strategy='majority')

X_train, y_train = undersample.fit_resample(X_train, y_train)
X_val, y_val = undersample.fit_resample(X_val, y_val)

In [ ]:
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
# y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)


## Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = MinMaxScaler().fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [ ]:
scaled_X_train = scaled_X_train.reshape(scaled_X_train.shape[0],int(scaled_X_train.shape[1]/4),4)
scaled_X_val = scaled_X_val.reshape(scaled_X_val.shape[0],int(scaled_X_val.shape[1]/4),4)
scaled_X_test = scaled_X_test.reshape(scaled_X_test.shape[0],int(scaled_X_test.shape[1]/4),4)

print(scaled_X_train.shape)
print(scaled_X_val.shape)
print(scaled_X_test.shape)

# CNN

In [ ]:
def get_model():
    leaky_relu_alpha = 0.3
    h_units = 64

    input1 = Input(shape=(scaled_X_train.shape[1],scaled_X_train.shape[2],1))
    
    y = Conv2D(h_units,(1,2),strides=(1,2))(input1)
#     y = LeakyReLU(leaky_relu_alpha)(y)
    y = Activation(tanh)(y)
    y = Conv2D(h_units,(1,2),strides=(1,2))(y)
#     y = LeakyReLU(leaky_relu_alpha)(y)
    y = Activation(tanh)(y)
    
    # build the inception module
    t1 = Conv2D(h_units, (1, 1), padding='same')(y)
#     t1 = LeakyReLU(alpha=leaky_relu_alpha)(t1)
    t1 = Activation(tanh)(t1)
    t1 = Conv2D(h_units, (5, 1), padding='same')(t1)
#     t1 = LeakyReLU(alpha=leaky_relu_alpha)(t1)
    t1 = Activation(tanh)(t1)

    t2 = Conv2D(h_units, (1, 1), padding='same')(y)
#     t2 = LeakyReLU(alpha=leaky_relu_alpha)(t2)
    t2 = Activation(tanh)(t2)
    t2 = Conv2D(h_units, (15, 1), padding='same')(t2)
#     t2 = LeakyReLU(alpha=leaky_relu_alpha)(t2)
    t2 = Activation(tanh)(t2)
  
    t3 = Conv2D(h_units, (1, 1), padding='same')(y)
#     t3 = LeakyReLU(alpha=leaky_relu_alpha)(t3)
    t3 = Activation(tanh)(t3)
    t3 = Conv2D(h_units, (30, 1), padding='same')(t3)
#     t3 = LeakyReLU(alpha=leaky_relu_alpha)(t3)
    t3 = Activation(tanh)(t3)

    t4 = Conv2D(h_units, (1, 1), padding='same')(y)
#     t4 = LeakyReLU(alpha=leaky_relu_alpha)(t4)
    t4 = Activation(tanh)(t4)
    t4 = Conv2D(h_units, (60, 1), padding='same')(t4)
#     t4 = LeakyReLU(alpha=leaky_relu_alpha)(t4)
    t4 = Activation(tanh)(t4)

    t5 = Conv2D(h_units, (1, 1), padding='same')(y)
#     t5 = LeakyReLU(alpha=leaky_relu_alpha)(t5)
    t5 = Activation(tanh)(t5)
    t5 = Conv2D(h_units, (120, 1), padding='same')(t5)
#     t5 = LeakyReLU(alpha=leaky_relu_alpha)(t5)
    t5 = Activation(tanh)(t5)


#     t3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(y)
#     t3 = Conv2D(64, (1, 1), padding='same')(t3)
#     t3 = LeakyReLU(alpha=0.01)(t3)

    all_time_periods = keras.layers.concatenate([t1, t2, t3, t4, t5], axis=3)
    
    y = Reshape((int(y.shape[1]), int(y.shape[3])))(y)
    y = Dropout(0.3)(y)

    # build the last LSTM layer
    conv_lstm = LSTM(h_units)(y)
    
    conv_lstm = Dropout(0.3)(conv_lstm)

    # build the output layer
    out = Dense(1,'sigmoid')(conv_lstm)
    
    model = Model(inputs=input1, outputs=out)
    adam = keras.optimizers.Adam(learning_rate=0.0005)
#     model.compile(optimizer=adam, loss='mean_squared_logarithmic_error', metrics=['mae'])
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

    return model
    

In [ ]:
m1 = get_model()


In [ ]:
m1.summary()

In [ ]:
earlystopper = EarlyStopping(monitor='val_loss',min_delta=0.01,patience=4,verbose=1,restore_best_weights=True)

In [ ]:
history = m1.fit(scaled_X_train,y_train,\
                   epochs=30,\
                   batch_size=128,\
                   validation_data=(scaled_X_val,y_val),\
                   verbose=1,\
                   callbacks=[earlystopper],\
                   shuffle=True
                  )

In [ ]:
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')
plt.legend()
plt.grid()
plt.show()

In [ ]:
preds = m1.predict(scaled_X_test)

In [ ]:
preds = np.argmax(preds,axis=1)

In [ ]:
results = pd.DataFrame({'preds':preds,'true':y_test})
# 'prev':X_test['PREV_1_pnls'].values
# results['predicted_pnl'] = results['preds']+results['prev']
# results['actual_pnl'] = results['true']+results['prev']

results

In [ ]:
results.abs().describe(np.arange(0,1,0.1))

In [ ]:
px.histogram(results.true)

In [ ]:
cm = confusion_matrix(y_test, preds, labels=[0,1],normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()